In [1]:
import torch
import torchvision.models as models
from torch import nn
import torchvision.transforms as transforms
from torchvision import datasets
import torch.optim as optim

In [2]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.485, .456,.406], std=[.229, .224,.226])
])

In [3]:
data  = datasets.EuroSAT(root = "./data",download=True, transform=transform)

In [4]:
from torch.utils.data.dataset import Subset
target_classes = ["River", "Industrial","Forest"]
class_indices = {data.classes[i]:i for i in range(len(data.classes))}
target_indices = [class_indices[cls] for cls in target_classes]

indices = [i for i, (_,label) in enumerate(data) if label in target_indices]

train_subset = Subset(data, indices)

In [5]:
print(f"Total images in subset: {len(train_subset)}")

Total images in subset: 8000


In [6]:
model = models.resnet50(pretrained = True)

for params in model.parameters():
  if "layer 4" or "fc" in params:
    params.requires_grad = True

num_classes = 3

model.fc = nn.Linear(model.fc.in_features, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW([
     {'params': model.layer4.parameters(), 'lr': 1e-5}, # Backbone fine-tuning
    {'params': model.fc.parameters(), 'lr': 1e-3} ,
 ], weight_decay=1e-2
)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
from torch.utils.data import random_split, DataLoader

# 1. Define Split Sizes (80% Train, 20% Test)
train_size = int(0.8 * len(train_subset))
test_size = len(train_subset) - train_size

# 2. Perform the Split
train_data, test_data = random_split(train_subset, [train_size, test_size])

# 3. Initialize Dataloaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Move model to device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)


In [8]:
label_map = {old_idx : new_idx for new_idx, old_idx in enumerate(target_indices)}

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


for epoch in range(5):
    model.train()
    for images, labels in train_loader:

        images = images.to(device)


        labels = torch.tensor([label_map[l.item()] for l in labels]).to(device)


        optimizer.zero_grad()
        loss = criterion(model(images), labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} Done. Loss: {loss.item():.4f}")

print("Done!")

Epoch 1 Done. Loss: 0.0145
Epoch 2 Done. Loss: 0.0022
Epoch 3 Done. Loss: 0.0041
Epoch 4 Done. Loss: 0.0004
Epoch 5 Done. Loss: 0.0001
Done!


In [13]:
model.eval()
all_preds = []
all_actuals = []

inv_label_map = {v: k for k, v in label_map.items()} # {0: 'River', 1: 'Industrial', 2: 'Forest'}

with torch.no_grad():
    for images, labels in test_loader:
        # 1. Prepare data
        images = images.to(device)
        actual_mapped = torch.tensor([label_map[l.item()] for l in labels]).to(device)

        # 2. Forward pass
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        # 3. Store results as names
        for p, a in zip(preds, actual_mapped):
            all_preds.append(inv_label_map[p.item()])
            all_actuals.append(inv_label_map[a.item()])

# 4. Print the first 10 results to check
print(f"{'Actual':<15} | {'Predicted':<15}")
print("-" * 35)
for i in range(100):
    print(f"{all_actuals[i]:<15} | {all_preds[i]:<15}")


Actual          | Predicted      
-----------------------------------
8               | 8              
1               | 1              
1               | 1              
8               | 8              
1               | 1              
8               | 8              
8               | 8              
8               | 8              
1               | 1              
8               | 8              
1               | 1              
4               | 4              
4               | 4              
1               | 1              
4               | 4              
4               | 4              
1               | 1              
8               | 8              
1               | 1              
8               | 8              
8               | 8              
4               | 4              
8               | 8              
8               | 8              
8               | 8              
4               | 4              
8               | 8              
8           

In [12]:
correct = sum(1 for p, a in zip(all_preds, all_actuals) if p == a)
total = len(all_actuals)
accuracy = (correct / total) * 100

print(f"Total Images Tested: {total}")
print(f"Correct Predictions: {correct}")
print(f"Final Accuracy: {accuracy:.2f}%")

Total Images Tested: 1600
Correct Predictions: 1595
Final Accuracy: 99.69%
